In [1]:
import os

from datetime import datetime, date
from pyspark.sql import SparkSession, Row

import pandas as pd

SPARK_MASTER = os.getenv("SPARK_MASTER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

In [2]:
# Add postgres jar
spark = (
    SparkSession.builder.appName("postgres-app")
    .master(SPARK_MASTER)
    .config("spark.jars", "/opt/spark/jars/postgresql-42.7.3.jar")
    .config("spark.executor.extraClassPath", "/opt/spark/jars/postgresql-42.7.3.jar")
    .config("spark.driver.extraClassPath", "/opt/spark/jars/postgresql-42.7.3.jar")
    .getOrCreate()
)

24/06/14 18:21:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = (
    spark.read.format("jdbc")
    .option("driver", "org.postgresql.Driver")
    .option("url", "jdbc:postgresql://docker-postgres:5432/postgres")
    .option("dbtable", "information_schema.tables")
    .option("user", "postgres")
    .option("password", POSTGRES_PASSWORD)
    .load()
)

df.printSchema()

root
 |-- table_catalog: string (nullable = true)
 |-- table_schema: string (nullable = true)
 |-- table_name: string (nullable = true)
 |-- table_type: string (nullable = true)
 |-- self_referencing_column_name: string (nullable = true)
 |-- reference_generation: string (nullable = true)
 |-- user_defined_type_catalog: string (nullable = true)
 |-- user_defined_type_schema: string (nullable = true)
 |-- user_defined_type_name: string (nullable = true)
 |-- is_insertable_into: string (nullable = true)
 |-- is_typed: string (nullable = true)
 |-- commit_action: string (nullable = true)



In [ ]:
df.show()

In [ ]:
spark.sql("SHOW DATABASES").show()

In [ ]:
spark.sql("SHOW TABLES").show()

In [ ]:
pandas_df = pd.DataFrame(
    {
        "a": [1, 2, 3],
        "b": [2.0, 3.0, 4.0],
        "c": ["string1", "string2", "string3"],
        "d": [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
        "e": [
            datetime(2000, 1, 1, 12, 0),
            datetime(2000, 1, 2, 12, 0),
            datetime(2000, 1, 3, 12, 0),
        ],
    }
)
df = spark.createDataFrame(pandas_df)
df

In [ ]:
df.show()

In [ ]:
df.collect()

In [ ]:
df = spark.createDataFrame(
    [
        Row(a=1, b=2.0, c="string1", d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
        Row(a=2, b=3.0, c="string2", d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
        Row(a=4, b=5.0, c="string3", d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)),
    ]
)
df

# Thrift

In [ ]:
# Connect to thrift
spark_thrift = (
    SparkSession.builder.appName("thrift_app")
    .master(SPARK_MASTER)
    .config("spark.hadoop.hive.metastore.uris", "spark://spark-master:10000")
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.legacy.createHiveTableByDefault", False)
    .getOrCreate()
)

In [ ]:
spark_thrift.conf.get("spark.sql.catalogImplementation")

In [ ]:
spark_thrift.conf.get("spark.sql.legacy.createHiveTableByDefault")

In [ ]:
spark_thrift.sql("create database hive_test;").show()

In [ ]:
spark_thrift.sql(
    "create table hive_example(a string, b int) partitioned by(c int);"
).show()